# 1 - Extração de recursos de texto

## 1.1 A Representação do modelo Bag Of Words(BOW)

## A Análise de Texto é um importante campo de aplicação para algoritmos de aprendizado de máquina. No entanto, os dados brutos, uma sequência de símbolos não podem ser alimentados diretamente para os próprios algoritmos, pois a maioria deles espera vetores de recursos numéricos com tamanho fixo, em vez de documentos de texto bruto com comprimento variável.

### .Para resolver isso, o scikit-learn fornece utilitários para as formas mais comuns de extrair recursos numéricos do conteúdo de texto, a saber:

### .Tokenizing strings e fornecendo um id inteiro para cada token possível, por exemplo, usando espaços em branco e pontuação como separadores de token.

### .Normalizando e Ponderando com tokens de importância decrescente que ocorrem na maioria das amostras/documentos.

## Nesse esquema, os recursos e as amostras são definidos da seguinte forma:

### .Cada frequência de ocorrência de token individual (normalizada ou não) é tratada como um recurso.

### .O vetor de todas as frequências de token para um determinado documento é considerado uma amostra multivariada.

## Um corpus de documentos pode assim ser representado por uma matriz com uma linha por documento e uma coluna por token (por exemplo, palavra) ocorrendo no corpus.

## Chamamos de vetorização o processo geral de transformar uma coleção de documentos de texto em vetores de recursos numéricos. Essa estratégia específica (tokenização, contagem e normalização) é chamada de representação Bag of Words ou “Bag of n-grams”. Os documentos são descritos por ocorrências de palavras, ignorando completamente as informações de posição relativa das palavras no documento.

## 1.1.2 Espasidade

### Por exemplo, uma coleção de 10.000 documentos de texto curto (como e-mails) usará um vocabulário com um tamanho da ordem de 100.000 palavras únicas no total, enquanto cada documento usará de 100 a 1.000 palavras únicas individualmente.

### Para poder armazenar tal matriz na memória, mas também para acelerar as operações algébricas de matriz/vetor, as implementações normalmente usarão uma representação esparsa, como as implementações disponíveis no pacote scipy.sparse.

## 1.1.3 Uso Comum do Vetorizador

### CountVectorizer implementa tokenização e contagem de ocorrências em uma única classe:

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

### Vamos usá-lo para tokenizar e contar as ocorrências de palavras de um corpus minimalista de documentos de texto, para isso vamos utilizar as seguintes frases :

In [6]:
frases = [
    "John likes",
    "likes to",
    "to watch",
    "watch movies",
    "Mary likes",
    "likes movies",
    "movies too",
]

In [8]:
vectorizer = CountVectorizer()
frs = vectorizer.fit_transform(frases)

### A configuração padrão tokeniza a string extraindo palavras de pelo menos 2 letras. A função específica que faz esta etapa pode ser solicitada explicitamente:

In [9]:
analyze = vectorizer.build_analyzer()

### verificando se há compatibilidade com as palavras nas frases.

In [12]:
analyze("Este documento é para a análise também.") == (['este', 'documento', 'para', 'análise', 'também'])

True

### Cada termo encontrado pelo analisador durante o ajuste recebe um índice inteiro único correspondente a uma coluna na matriz resultante. Essa interpretação das colunas pode ser recuperada da seguinte forma:

In [13]:
vectorizer.get_feature_names_out()

array(['john', 'likes', 'mary', 'movies', 'to', 'too', 'watch'],
      dtype=object)

In [14]:
frs.toarray()

array([[1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0]])

### O mapeamento inverso do nome do recurso para o índice da coluna é armazenado no atributo vocabulário_ do vetorizador:

In [15]:
vectorizer.vocabulary_.get('mary')

2

### Portanto, palavras que não foram vistas no corpus de treinamento serão completamente ignoradas em futuras chamadas ao método transform:

In [16]:
vectorizer.transform(['Something completely new.']).toarray()

array([[0, 0, 0, 0, 0, 0, 0]])

### Observe que no corpus anterior, o primeiro e o último documentos têm exatamente as mesmas palavras, portanto, são codificados em vetores iguais. Em particular, perdemos a informação de que o último documento é uma forma interrogativa. Para preservar algumas das informações de pedidos locais, podemos extrair 2-grams de palavras além dos 1-grams (palavras individuais):

In [17]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),
token_pattern=r'\b\w+\b', min_df=1)

In [18]:
analyze = bigram_vectorizer.build_analyzer()

In [19]:
analyze('Bi-gramas são legais!') == (
     ['bi', 'gramas', 'são', 'legais', 'bi gramas', 'gramas são', 'são legais'])

True

### O vocabulário extraído por este vetorizador é, portanto, muito maior e agora pode resolver ambiguidades codificadas em padrões de posicionamento local:

In [20]:
frs_1 = bigram_vectorizer.fit_transform(frases).toarray()
frs_1

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]])

### Em particular, a forma interrogativa “Este” está presente apenas no último documento:

In [21]:
feature_index = bigram_vectorizer.vocabulary_.get('este')

In [22]:
frs_1[:, feature_index]

array([[[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]],

       [[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]]])

### Em um corpus de texto grande, algumas palavras estarão muito presentes (por exemplo, “the”, “a”, “is” em inglês), portanto, carregam muito pouca informação significativa sobre o conteúdo real do documento. Se fôssemos alimentar os dados de contagem direta diretamente para um classificador, esses termos muito frequentes sombreariam as frequências de termos mais raros, porém mais interessantes.

### Para reponderar os recursos de contagem em valores de ponto flutuante adequados para uso por um classificador, é muito comum usar a transformada tf–idf.

### Tf significa termo-frequência enquanto tf–idf significa o "produto" (termo-frequência vezes inversa do documento-frequência) 
### tf-idf(t,d) = tf(t,d) * idf(t).

### Essa normalização é implementada pela classe TfidfTransformer:

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(smooth_idf=False)

## Vamos dar um exemplo com as seguintes contagens. O primeiro termo está presente 100% do tempo, portanto, não é muito interessante. Os outros dois recursos apenas em menos de 50% do tempo, portanto, provavelmente mais representativos do conteúdo dos documentos:

In [24]:
counts = [[3, 0, 1],
           [2, 0, 0],
           [3, 0, 0],
           [4, 0, 0],
           [3, 2, 0],
           [3, 0, 2]]

tfidf = transformer.fit_transform(counts)
tfidf

<6x3 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

### Cada linha é normalizada para ter a norma euclidiana unitária:

### Além disso, o parâmetro padrão smooth_idf=True adiciona “1” ao numerador e denominador como se um documento extra fosse visto contendo todos os termos da coleção exatamente uma vez, o que evita divisões zero:

In [25]:
transformer = TfidfTransformer()
transformer.fit_transform(counts).toarray()

array([[0.85151335, 0.        , 0.52433293],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.55422893, 0.83236428, 0.        ],
       [0.63035731, 0.        , 0.77630514]])

### Os pesos de cada recurso calculados pela chamada do método fit são armazenados em um atributo de modelo:

In [26]:
transformer.idf_

array([1.        , 2.25276297, 1.84729786])

### Como tf–idf é muito usado para recursos de texto, existe também outra classe chamada TfidfVectorizer que combina todas as opções de CountVectorizer e TfidfTransformer em um único modelo:

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(frases)

<7x7 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

### Embora a normalização tf–idf seja frequentemente muito útil, pode haver casos em que os marcadores de ocorrência binários possam oferecer melhores recursos. Isso pode ser feito usando o parâmetro binário de CountVectorizer. Em particular, alguns estimadores como Bernoulli Naive Bayes modelam explicitamente variáveis ​​aleatórias booleanas discretas. Além disso, é provável que textos muito curtos tenham valores tf–idf ruidosos, enquanto as informações de ocorrência binária são mais estáveis.

### Como de costume, a melhor maneira de ajustar os parâmetros de extração de recursos é usar uma pesquisa de grade com validação cruzada, por exemplo, canalizando o extrator de recursos com um classificador:

### Veja também amostra de Pipeline para extração e avaliação de recursos de texto!